2025-05-18 20:59:43.056274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747601983.255479      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747601983.317877      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


⚠ tensorflow-addons not available. Falling back to Adam optimizer and skipping random_cutout.

✅ TensorFlow version: 2.18.0
✅ GPU device name: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ Using GPU(s): ['/physical_device:GPU:0']
Contents of /kaggle/input/kmapping-csv:
['converted_labels.csv']

✅ First 5 rows of converted_labels.csv:
       tweet_id              image_id   text_info  text_info_conf  image_info  \
0  9.300000e+17  929989361453621249_0      damage          1.0000      damage   
1  9.300000e+17  929989388863508480_0  non-damage          0.7535  non-damage   
2  9.300000e+17  929989414826024960_0      damage          1.0000  non-damage   
3  9.300000e+17  929989443913691136_0      damage          1.0000      damage   
4  9.300000e+17  929989443913691136_1      damage          1.0000      damage   

   image_info_conf                  text_human  text_human_conf  \
0           0.6513      injured_or_dead_people           1.0000   
1           1.0000  

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


  Subdirectory: mexico_earthquake (256 files)
Directory: train (7 files)
  Subdirectory: srilanka_floods (742 files)
  Subdirectory: iraq_iran_earthquake (435 files)
  Subdirectory: Hurricane_Harvey (3386 files)
  Subdirectory: Hurricane_Maria (3447 files)
  Subdirectory: Hurricane_Irma (3446 files)
  Subdirectory: California_Wildfires (1235 files)
  Subdirectory: mexico_earthquake (1024 files)

Scanning train directory: /kaggle/input/split-data/content/split_data/
Checking folder: /kaggle/input/split-data/content/split_data/train/srilanka_floods
Found 742 images in srilanka_floods
Checking folder: /kaggle/input/split-data/content/split_data/train/iraq_iran_earthquake
Found 435 images in iraq_iran_earthquake
Checking folder: /kaggle/input/split-data/content/split_data/train/Hurricane_Harvey
Found 3386 images in Hurricane_Harvey
Checking folder: /kaggle/input/split-data/content/split_data/train/Hurricane_Maria
Found 3447 images in Hurricane_Maria
Checking folder: /kaggle/input/split-dat

I0000 00:00:1747602074.242334      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,0

 Total params: 14,979,905 (57.14 MB)

 Trainable params: 13,243,393 (50.52 MB)

 Non-trainable params: 1,736,512 (6.62 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1747602085.141712      96 service.cc:148] XLA service 0x78e8cc014de0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747602085.142536      96 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1747602085.811256      96 cuda_dnn.cc:529] Loaded cuDNN version 90300


  2/936 ━━━━━━━━━━━━━━━━━━━━ 1:12 78ms/step - accuracy: 0.4375 - loss: 5.9856 - macro_f1_score: 0.5070  

I0000 00:00:1747602093.435235      96 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


936/936 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 0.6352 - loss: 4.1457 - macro_f1_score: 0.5017
Epoch 1
Train - Loss: 2.9883, Acc: 0.6554, Macro F1: 0.5013
Test  - Loss: 1.4873, Acc: 0.7204, Prec: 0.7296, Rec: 0.7301, F1: 0.7298, Macro F1: 0.5201
Saving model with improved TEST Macro F1: 0.5201
936/936 ━━━━━━━━━━━━━━━━━━━━ 323s 331ms/step - accuracy: 0.6352 - loss: 4.1445 - macro_f1_score: 0.5017
Epoch 2/20
936/936 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.6962 - loss: 1.2909 - macro_f1_score: 0.5040
Epoch 2
Train - Loss: 1.1475, Acc: 0.6960, Macro F1: 0.5031
Test  - Loss: 1.0037, Acc: 0.6891, Prec: 0.6428, Rec: 0.8981, F1: 0.7493, Macro F1: 0.4964
936/936 ━━━━━━━━━━━━━━━━━━━━ 220s 235ms/step - accuracy: 0.6962 - loss: 1.2907 - macro_f1_score: 0.5040
Epoch 3/20
936/936 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.7131 - loss: 0.8707 - macro_f1_score: 0.5023
Epoch 3
Train - Loss: 0.8256, Acc: 0.7127, Macro F1: 0.5021
Test  - Loss: 0.7209, Acc: 0.7418, Prec: 0.7370, Rec:

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
import glob
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils import class_weight

# ✅ Install tensorflow-addons (run this in a separate Kaggle cell if needed)
# !pip install tensorflow-addons

try:
    import tensorflow_addons as tfa
    tfa_available = True
except ImportError:
    print("⚠ tensorflow-addons not available. Falling back to Adam optimizer and skipping random_cutout.")
    tfa_available = False

# ✅ Custom Cutout Implementation
def custom_cutout(image):
    if not tfa_available:
        return image
    h, w = image.shape[:2]
    mask_size = tf.random.uniform([], 0.1, 0.3, dtype=tf.float32) * tf.cast(tf.minimum(h, w), tf.float32)
    mask_size = tf.cast(mask_size, tf.int32)
    cx = tf.random.uniform([], 0, tf.cast(w, tf.float32), dtype=tf.float32)
    cy = tf.random.uniform([], 0, tf.cast(h, tf.float32), dtype=tf.float32)
    cx = tf.cast(cx, tf.int32)
    cy = tf.cast(cy, tf.int32)
    x1 = tf.clip_by_value(cx - mask_size // 2, 0, w)
    x2 = tf.clip_by_value(cx + mask_size // 2, 0, w)
    y1 = tf.clip_by_value(cy - mask_size // 2, 0, h)
    y2 = tf.clip_by_value(cy + mask_size // 2, 0, h)
    mask = tf.ones([h, w, 3])
    mask = tf.tensor_scatter_nd_update(
        mask,
        tf.stack([tf.range(y1, y2), tf.range(x1, x2)], axis=1),
        tf.zeros([(y2 - y1) * (x2 - x1) * 3])
    )
    return image * mask

# ✅ GPU Setup
print("\n✅ TensorFlow version:", tf.__version__)
print("✅ GPU device name:", tf.config.list_physical_devices('GPU'))
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("✅ Using GPU(s):", [gpu.name for gpu in physical_devices])
else:
    print("⚠ Using CPU")

# ✅ Load CSV file for remapping
csv_path = "/kaggle/input/kmapping-csv/converted_labels.csv"
print("Contents of /kaggle/input/kmapping-csv:")
print(os.listdir("/kaggle/input/kmapping-csv"))
mapping_df = pd.read_csv(csv_path)

# Debug CSV contents
print("\n✅ First 5 rows of converted_labels.csv:")
print(mapping_df.head())
print("\n✅ Column names:", mapping_df.columns.tolist())
print("\n✅ Unique image_id count:", mapping_df['image_id'].nunique())
print("\n✅ Unique image_info values:", mapping_df['image_info'].str.lower().unique())

# ✅ Function to parse image_info
def parse_image_info(image_info):
    image_info = str(image_info).lower().strip()
    if image_info == 'damage':
        return '1'  # Damage
    elif image_info == 'non-damage':
        return '0'  # Non-Damage
    else:
        return None  # Unknown, will be filtered out

# ✅ Create damage mapping
mapping_df['class'] = mapping_df['image_info'].apply(parse_image_info)
print("\n✅ Parsed class distribution:")
print(mapping_df['class'].value_counts(dropna=False))
invalid_info = mapping_df[mapping_df['class'].isnull()][['image_id', 'image_info']]
if not invalid_info.empty:
    print("\n✅ Invalid image_info entries:")
    print(invalid_info.head())
    invalid_info.to_csv(os.path.join('/kaggle/working/', 'invalid_image_info.csv'), index=False)
mapping_df = mapping_df[mapping_df['class'].notnull()]
damage_mapping = dict(zip(mapping_df['image_id'], mapping_df['class']))
print(f"\n✅ Damage mapping created with {len(damage_mapping)} entries")

# ✅ Utility function: Create DataFrame for train or test
def create_dataframe(data_dir, mapping, split='train'):
    filepaths = []
    labels = []
    missing_images = []
    print(f"\nScanning {split} directory: {data_dir}")
    split_path = os.path.join(data_dir, split)
    for disaster_name in os.listdir(split_path):
        folder_path = os.path.join(split_path, disaster_name)
        if not os.path.isdir(folder_path):
            continue
        print(f"Checking folder: {folder_path}")
        files = glob.glob(os.path.join(folder_path, '*.[jJ][pP][gG]')) + \
                glob.glob(os.path.join(folder_path, '*.[jJ][pP][eE][gG]')) + \
                glob.glob(os.path.join(folder_path, '*.[pP][nN][gG]'))
        print(f"Found {len(files)} images in {disaster_name}")
        if len(files) == 0:
            all_files = glob.glob(os.path.join(folder_path, '*'))
            print(f"All files in {disaster_name}: {all_files}")
        for file in files:
            filename = os.path.basename(file)
            image_id = os.path.splitext(filename)[0]
            if image_id in mapping:
                filepaths.append(file)
                labels.append(mapping[image_id])
            else:
                print(f"Warning: No mapping found for image_id {image_id} in {file}")
                missing_images.append((image_id, file))
    missing_images_path = os.path.join('/kaggle/working/', f'missing_images_{split}.txt')
    with open(missing_images_path, 'w') as f:
        for image_id, file in missing_images:
            f.write(f"{image_id},{file}\n")
    print(f"✅ Total images skipped in {split}: {len(missing_images)}")
    df = pd.DataFrame({'filename': filepaths, 'class': labels})
    return df

# ✅ Data Paths
base_path = "/kaggle/working/"
input_data_path = "/kaggle/input/split-data/content/split_data/"
os.makedirs(base_path, exist_ok=True)

# ✅ Debug Directory Contents
print("\nContents of /kaggle/input/split-data/content/split_data/:")
for item in os.listdir(input_data_path):
    item_path = os.path.join(input_data_path, item)
    if os.path.isdir(item_path):
        num_files = len(glob.glob(os.path.join(item_path, '*')))
        print(f"Directory: {item} ({num_files} files)")
        for sub_item in os.listdir(item_path):
            sub_item_path = os.path.join(item_path, sub_item)
            if os.path.isdir(sub_item_path):
                num_sub_files = len(glob.glob(os.path.join(sub_item_path, '*')))
                print(f"  Subdirectory: {sub_item} ({num_sub_files} files)")
    else:
        print(f"File: {item}")

# ✅ Remove hidden files
os.system("find /kaggle/input/split-data/ -name '.*' -delete")

# ✅ Create Train and Test DataFrames (80/20 split assumed in dataset)
train_df = create_dataframe(input_data_path, damage_mapping, split='train')
test_df = create_dataframe(input_data_path, damage_mapping, split='test')

print("\nTraining DataFrame distribution:")
print(train_df['class'].value_counts())
print("\nTest DataFrame distribution:")
print(test_df['class'].value_counts())

# Check if DataFrames are empty
if train_df.empty or test_df.empty:
    raise ValueError(
        "One or both DataFrames are empty. Please check the following:\n"
        "1. The dataset path '/kaggle/input/split-data/content/split_data/' contains 'train' and 'test' subfolders.\n"
        "2. The CSV file contains valid image_id and image_info entries ('damage' or 'non-damage').\n"
        "3. The image_id in the CSV matches the filenames in the dataset.\n"
        "4. The dataset is correctly uploaded to Kaggle under '/kaggle/input/split-data/content/split_data/'.\n"
        "5. The folder structure is: /kaggle/input/split-data/content/split_data/[train|test]/<disaster_name>/*.jpg (or .jpeg, .png).\n"
        "6. Check '/kaggle/working/missing_images_{split}.txt' for unmapped images."
    )

# ✅ Oversample minority class
minority_class = '0' if (train_df['class'] == '0').sum() < (train_df['class'] == '1').sum() else '1'
minority_df = train_df[train_df['class'] == minority_class]
oversampled_df = minority_df.sample(n=int(0.2 * len(minority_df)), replace=True, random_state=42)
train_df = pd.concat([train_df, oversampled_df]).sample(frac=1, random_state=42).reset_index(drop=True)
print("\nOversampled Training DataFrame distribution:")
print(train_df['class'].value_counts())

# ✅ Compute Class Weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['class'].astype(int)),
    y=train_df['class'].astype(int)
)
class_weight_dict = dict(enumerate(class_weights))
print("\n✅ Class weights:", class_weight_dict)

# ✅ Create Data Generators
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],
    channel_shift_range=20.0,
    fill_mode='nearest',
    preprocessing_function=custom_cutout if tfa_available else None
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True,
    classes=['0', '1']
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,
    classes=['0', '1']
)

# ✅ Define Custom Macro F1 Score Metric
class MacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, name='macro_f1_score', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')
        self.true_negatives = self.add_weight(name='tn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(y_pred)
        tp = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
        tn = tf.reduce_sum(tf.cast((1 - y_true) * (1 - y_pred), tf.float32))
        fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, tf.float32))
        fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), tf.float32))
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)
        self.true_negatives.assign_add(tn)

    def result(self):
        precision_pos = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall_pos = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1_pos = 2 * (precision_pos * recall_pos) / (precision_pos + recall_pos + tf.keras.backend.epsilon())
        precision_neg = self.true_negatives / (self.true_negatives + self.false_negatives + tf.keras.backend.epsilon())
        recall_neg = self.true_negatives / (self.true_negatives + self.false_positives + tf.keras.backend.epsilon())
        f1_neg = 2 * (precision_neg * recall_neg) / (precision_neg + recall_neg + tf.keras.backend.epsilon())
        return (f1_pos + f1_neg) / 2

    def reset_states(self):
        self.true_positives.assign(0.0)
        self.false_positives.assign(0.0)
        self.false_negatives.assign(0.0)
        self.true_negatives.assign(0.0)

# ✅ Build VGG16-based model
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = True
for layer in base_model.layers[:-8]:  # Fine-tune last 8 layers
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = BatchNormalization()(x)
x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=output)

# ✅ Model Summary
model.summary()

# ✅ Cosine Decay Learning Rate Schedule
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=30 * len(train_generator), alpha=1e-6
)

# ✅ Compile with Ranger optimizer or Adam fallback
if tfa_available:
    optimizer = tfa.optimizers.RectifiedAdam(learning_rate=lr_schedule)
    optimizer = tfa.optimizers.Lookahead(optimizer)
else:
    optimizer = Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', MacroF1Score()])

# ✅ Callbacks
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

class BestTestMacroF1ModelSaver(Callback):
    def __init__(self, test_data, save_path=os.path.join(base_path, 'best_test_macro_f1_model.keras')):
        super(BestTestMacroF1ModelSaver, self).__init__()
        self.test_data = test_data
        self.best_test_macro_f1 = 0.0
        self.save_path = save_path
        self.best_epoch = 0
        self.best_metrics = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        train_loss = logs.get('loss', 0.0)
        train_acc = logs.get('accuracy', 0.0)
        train_f1 = logs.get('macro_f1_score', 0.0)

        # Evaluate on test set
        self.test_data.reset()
        y_true_test = self.test_data.classes
        y_pred_test = self.model.predict(self.test_data, verbose=0)
        y_pred_test_classes = (y_pred_test > 0.5).astype(int).reshape(-1)

        # Compute test metrics
        test_loss, test_acc, test_macro_f1 = self.model.evaluate(self.test_data, verbose=0)
        test_prec = precision_score(y_true_test, y_pred_test_classes)
        test_rec = recall_score(y_true_test, y_pred_test_classes)
        test_f1 = f1_score(y_true_test, y_pred_test_classes)

        # Print metrics for current epoch
        print(f"\nEpoch {epoch+1}")
        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Macro F1: {train_f1:.4f}")
        print(f"Test  - Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, Prec: {test_prec:.4f}, Rec: {test_rec:.4f}, F1: {test_f1:.4f}, Macro F1: {test_macro_f1:.4f}")

        # Save model and metrics if test Macro F1 improves
        if test_macro_f1 > self.best_test_macro_f1:
            print(f"Saving model with improved TEST Macro F1: {test_macro_f1:.4f}")
            self.best_test_macro_f1 = test_macro_f1
            self.best_epoch = epoch + 1
            self.best_metrics = {
                'train_loss': train_loss,
                'train_acc': train_acc,
                'train_macro_f1': train_f1,
                'test_loss': test_loss,
                'test_acc': test_acc,
                'test_prec': test_prec,
                'test_rec': test_rec,
                'test_f1': test_f1,
                'test_macro_f1': test_macro_f1
            }
            self.model.save(self.save_path)

    def on_train_end(self, logs=None):
        print(f"\nFinal Result - Best Epoch: {self.best_epoch}")
        print("Best Test Metrics:")
        print(f"Accuracy: {self.best_metrics['test_acc']:.4f}")
        print(f"Precision: {self.best_metrics['test_prec']:.4f}")
        print(f"Recall: {self.best_metrics['test_rec']:.4f}")
        print(f"F1 Score: {self.best_metrics['test_f1']:.4f}")
        print(f"Macro F1 Score: {self.best_metrics['test_macro_f1']:.4f}")
        print(f"(Train - Loss: {self.best_metrics['train_loss']:.4f}, Acc: {self.best_metrics['train_acc']:.4f}, Macro F1: {self.best_metrics['train_macro_f1']:.4f})")
        print(f"(Test Loss: {self.best_metrics['test_loss']:.4f})")

# ✅ Train the model
history = model.fit(
    train_generator,
    epochs=20,
    class_weight=class_weight_dict,
    callbacks=[early_stop, BestTestMacroF1ModelSaver(test_generator)],
    verbose=1
)

# ✅ Save Final Model
final_model_path = os.path.join(base_path, "final_trained_model.keras")
model.save(final_model_path)
print(f"✅ Final model saved to: {final_model_path}")

# ✅ Evaluation on Test Set
model = load_model(
    os.path.join(base_path, "best_test_macro_f1_model.keras"),
    custom_objects={'MacroF1Score': MacroF1Score}
)
test_generator.reset()
y_true_test = test_generator.classes
y_pred_test = model.predict(test_generator, verbose=1)
y_pred_test_classes = (y_pred_test > 0.5).astype(int).reshape(-1)

cm = confusion_matrix(y_true_test, y_pred_test_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Damage', 'Damage'], yticklabels=['Non-Damage', 'Damage'])
plt.title('Confusion Matrix (Best Epoch - Test Set)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig(os.path.join(base_path, 'confusion_matrix.png'))

test_acc = accuracy_score(y_true_test, y_pred_test_classes)
test_prec = precision_score(y_true_test, y_pred_test_classes)
test_rec = recall_score(y_true_test, y_pred_test_classes)
test_f1 = f1_score(y_true_test, y_pred_test_classes)
test_macro_f1 = f1_score(y_true_test, y_pred_test_classes, average='macro')

print("\n✅ Test Set Evaluation (Best Epoch Confirmation):")
print(f"Accuracy: {test_acc:.4f} | Precision: {test_prec:.4f} | Recall: {test_rec:.4f} | F1 Score: {test_f1:.4f} | Macro F1: {test_macro_f1:.4f}")
print("\n🧾 Classification Report (Test):")
print(classification_report(y_true_test, y_pred_test_classes, target_names=['Non-Damage', 'Damage']))

# ✅ Visualize Metrics
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(history.history['macro_f1_score'], label='Train Macro F1')
plt.title('Macro F1 Score')
plt.xlabel('Epoch')
plt.ylabel('Macro F1')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(base_path, 'training_metrics.png'))

# ✅ Visualize Misclassified Images
misclassified_idx = np.where(y_true_test != y_pred_test_classes)[0]
if len(misclassified_idx) > 0:
    print("\n✅ Visualizing 5 misclassified images:")
    plt.figure(figsize=(15, 3))
    for i, idx in enumerate(misclassified_idx[:5]):
        img_path = test_df.iloc[idx]['filename']
        img = load_img(img_path, target_size=(224, 224))
        plt.subplot(1, 5, i+1)
        plt.imshow(img)
        plt.title(f"True: {y_true_test[idx]}, Pred: {y_pred_test_classes[idx]}")
        plt.axis('off')
    plt.savefig(os.path.join(base_path, 'misclassified_images.png'))